# Getting started

## Before you start

### Custom Dataset
If you want to run prompt optimization on your own dataset, follow these steps:

1. Create a folder.
1. Create a .txt file in the folder named "prompts.txt". It should contain 8-12 initial prompts from where you can start the optimization. Add line breaks between each of the prompts
1. Create two .txt files in another folder, which contain the dev set "dev.txt" and test set "test.txt" of your data points. Convert the classes of your file into integers. 
Make sure to seperate the input from the expected output with a tab!
1. Create a description.json file that contains a dictionary, specifying:
    - "seed": the folder in which you find the dev and test files
    - "init_prompts": the name of the .txt file pointing to the prompts
    - "description": A short description of your task, that is fed to the meta-llm in order to optimize the prompts. 
    (TIP: Include "The class mentioned first in the response of the LLM will be the prediction." in the description if this is how you evaluate the models responses)
    - "classes": A list of the names of the classes you are trying to predict

You can find examples of how this needs to be set up in our repo at data_sets/

## Installs

In [1]:
! pip install promptolution

## Imports

In [7]:
from promptolution.helpers import run_experiment
from promptolution.config import Config

## set up llms, predictor, tasks and optimizer

In [8]:
token = open("../deepinfratoken.txt", "r").read()

In [9]:
config = Config(
    task_name="agnews",
    ds_path="../data_sets/cls/agnews/",
    n_steps=3,
    optimizer="evopromptga",
    meta_llm="meta-llama/Meta-Llama-3-8B-Instruct",
    evaluation_llm="meta-llama/Meta-Llama-3-8B-Instruct",
    downstream_llm="meta-llama/Meta-Llama-3-8B-Instruct",
    api_token=token,
)

In [10]:
df = run_experiment(config)

In [11]:
df

,prompt,score
0,Classify the news story into one of the follow...,0.95
7,You will be required to classify a news articl...,0.90
11,"Classify the topic of the following news as ""W...",0.90
12,"Classify news articles into categories (World,...",0.90
4,Classify the given news article into one of th...,0.85
6,Your job is to determine whether a news articl...,0.85
13,Categorize the provided news article according...,0.85
1,Categorize the news article into one of four c...,0.80
2,Your responsibility is to accurately categoriz...,0.80
3,Identify the primary theme of a news article a...,0.80
